# 3 hr Statistics

This script is used to show the 3-hr statistics such as max and min

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import gc

In [2]:
ds = xr.open_dataset("../nc_files/f09.nc")

# create masks based on percentile for chi
ds["opt1_per"] = ds["s1_opt1"]/(ds["s1_opt1"]+ds["s2_opt1"])
ds["hyg_per"] = ds["s1_hyg"]/(ds["s1_hyg"]+ds["s2_hyg"])
ds["Mass_all"] = ds["Mass_bc"]+ds["Mass_dst"]+ds["Mass_ncl"]+ds["Mass_pom"]+ds["Mass_soa"]+ds["Mass_so4"]
for var in ["bc","dst","ncl","pom","soa","so4"]:
    ds[var+"_per"]=ds["Mass_"+var]/ds["Mass_all"]

ds=ds[["chi_abd","chi_opt1","chi_hyg",
       "opt1_per","hyg_per",
       "bc_per","dst_per","ncl_per","pom_per","soa_per","so4_per",
       "Mass_bc","Mass_dst","Mass_ncl","Mass_pom","Mass_soa","Mass_so4"]]

ds["polluted"] = ds["Mass_bc"]+ds["Mass_pom"]+ds["Mass_soa"]+ds["Mass_so4"]

ds["mask"] = xr.DataArray(np.ones(ds["bc_per"].shape), 
                          coords = [ds.lat, ds.lon, ds.season],
                          dims = ["lat","lon","season"])

ds["mask_chi_abd"] = ds["mask"].where((ds["bc_per"]<0.975)&(ds["dst_per"]<0.975)\
                                    &(ds["ncl_per"]<0.975)&(ds["pom_per"]<0.975)\
                                    &(ds["soa_per"]<0.975)&(ds["so4_per"]<0.975))\
                                    .notnull().squeeze()

ds["mask_chi_opt1"] = ds["mask"].where((ds["bc_per"]>0.01)&(ds["bc_per"]<0.99))\
                                .notnull().squeeze()

ds["mask_chi_hyg"] = ds["mask"].where(((ds["bc_per"]+ds["dst_per"]+ds["pom_per"])>0.025)\
                                &((ds["bc_per"]+ds["dst_per"]+ds["pom_per"])<0.975))\
                               .notnull().squeeze()

ds["mask_high_polluted"] = ds["mask"].where((ds["polluted"]>(ds["polluted"].quantile(0.90))))\
                                .notnull().squeeze()

## report the std of chi_hyg using the 3 hrs data

In [3]:
month_dict = {}
month_dict["DJF"] = [12,1,2]
month_dict["JJA"] = [6,7,8]
path = "/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/"

In [4]:
def season_stat(path, chi, month_dict, season, ds_mask):
    for month in month_dict[season]:
        print(path+str(month).zfill(2)+".nc")
        ds_month = xr.open_dataset(path+str(month).zfill(2)+".nc")[chi]\
                   .where(ds["mask_chi_abd"].sel(season="DJF"))
        print("min:", np.float(ds_month.quantile(0.0005)))
        print("max:", np.float(ds_month.quantile(0.9995)))
        print("std:", np.float(ds_month.std()))
        print("mean", np.float(ds_month.mean()))
        print("median:", np.float(ds_month.median()))
        print("\n")

## DJF

In [5]:
for chi in ["chi_abd","chi_opt1","chi_hyg"]:
    print(chi)
    season_stat(path, chi, month_dict, "DJF", ds)
    print("####################")

chi_abd
/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/12.nc
min: 0.033156691893935206
max: 0.9577084824740939
std: 0.14381402730941772
mean 0.6372407078742981
median: 0.6474694013595581


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/01.nc
min: 0.04624362464249134
max: 0.9552862644195557
std: 0.1436331868171692
mean 0.6359810829162598
median: 0.6455718278884888


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/02.nc
min: 0.049327526435256
max: 0.9600391000807292
std: 0.1440524011850357
mean 0.6571996212005615
median: 0.6743100881576538


####################
chi_opt1
/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/12.nc
min: 0.12407775884866715
max: 0.9330786451101383
std: 0.16679473221302032
mean 0.5812399387359619
median: 0.5973672866821289


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/01.nc
min: 0.11330815133452415
max: 0.9439077437520058
std: 0.16872049868106842
mean 0.5

## JJA

In [6]:
for chi in ["chi_abd","chi_opt1","chi_hyg"]:
    print(chi)
    season_stat(path, chi, month_dict, "JJA", ds)
    print("####################")

chi_abd
/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/06.nc
min: 0.04158608508110046
max: 1.0
std: 0.15748195350170135
mean 0.6900191903114319
median: 0.7076271176338196


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/07.nc
min: 0.10122249145805837
max: 1.0
std: 0.14636391401290894
mean 0.6955495476722717
median: 0.7136132717132568


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/08.nc
min: 0.10593466141819954
max: 1.0
std: 0.14703211188316345
mean 0.6998758912086487
median: 0.7288135290145874


####################
chi_opt1
/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/06.nc
min: 0.21881930787861348
max: 0.9860904015004655
std: 0.1503339260816574
mean 0.6646656394004822
median: 0.6868863701820374


/data/keeling/a/zzheng25/a/PartMC-MAM4/PartMC-ML-updated-predictions/07.nc
min: 0.23222560408711435
max: 0.9820152339935326
std: 0.1374751329421997
mean 0.663556694984436
median: 0.6864159107208252


